# How to create an LGST-only report
This example shows how you can create a HTML report from just the results of running *linear GST* (LGST).  This can be useful when you want to get a rough estimate of your gates quickly, as LGST is takes substantially less data and computation time compared with long-sequence GST.  This example is modeled after Tutorial 0.

In [1]:
#Make print statements compatible with Python 2 and 3
from __future__ import print_function

#Import the pygsti module (always do this) and the standard XYI gate set
import pygsti
from pygsti.construction import std1Q_XYI

# 1) get the target GateSet
gs_target = std1Q_XYI.gs_target

# 2) get the building blocks needed to specify which gate sequences are needed
prep_fiducials, meas_fiducials = std1Q_XYI.prepStrs, std1Q_XYI.effectStrs
germs = pygsti.construction.gatestring_list([(), ('Gi',), ('Gx',), ('Gy',)] ) # just the single gates
maxLengths = [1] # we only need maxLength == 1 when doing LGST

# 3) generate "fake" data from a depolarized version of gs_target
gs_datagen = gs_target.depolarize(gate_noise=0.1, spam_noise=0.001)
listOfExperiments = pygsti.construction.make_lsgst_experiment_list(
    gs_target, prep_fiducials, meas_fiducials, germs, maxLengths)
ds = pygsti.construction.generate_fake_data(gs_datagen, listOfExperiments, nSamples=1000,
                                            sampleError="binomial", seed=1234)

#Note: from listOfExperiments we can also create an empty dataset file
# which has columns of zeros where actual data should go. 
pygsti.io.write_empty_dataset("example_files/LGSTReportDataTemplate.txt", listOfExperiments,
                              "## Columns = 0 count, 1 count")
# After replacing the zeros with actual data, the data set can be 
# loaded back into pyGSTi using the line below and used in the rest
# of this tutorial. 
#ds = pygsti.io.load_dataset("example_files/LGSTReportDataTemplate.txt")

print("Only %d sequences are required!" % len(listOfExperiments))

Only 92 sequences are required!


In [2]:
#Run GST and create a report
# Key part is setting 'objective' advanced option == "lgst"
# You can also eliminate gauge optimization step by setting gaugeOptSuite="none"
results = pygsti.do_stdpractice_gst(ds, gs_target, prep_fiducials, meas_fiducials, germs, maxLengths,
                                    advancedOptions={'all': {'objective': 'lgst'}},
                                    gaugeOptSuite="single", modes="TP")
pygsti.report.create_standard_report(results, filename="example_files/LGSTonlyReport",
                                    title="LGST-only Example Report", verbosity=2)

-- Std Practice:  Iter 1 of 1  (TP) --: 
  --- Gate Sequence Creation ---
  --- LGST ---
  -- Performing 'single' gauge optimization on TP estimate --
*** Creating workspace ***
*** Generating switchboard ***
Found standard clifford compilation from std1Q_XYI
*** Generating tables ***
  targetSpamBriefTable                          took 0.017485 seconds
  targetGatesBoxTable                           took 0.017166 seconds
  datasetOverviewTable                          took 0.002498 seconds
  bestGatesetSpamParametersTable                took 0.000411 seconds
  bestGatesetSpamBriefTable                     took 0.02322 seconds
  bestGatesetSpamVsTargetTable                  took 0.047793 seconds
  bestGatesetGaugeOptParamsTable                took 0.000624 seconds
  bestGatesetGatesBoxTable                      took 0.039462 seconds
  bestGatesetChoiEvalTable                      took 0.034002 seconds
  bestGatesetDecompTable                        took 0.030146 seconds
  bestGatesetEv

Click to open the file [example_files/LGSTonlyReport/main.html](example_files/LGSTonlyReport/main.html) in your browser to view the report.